app is a text classification model that uses a pre-trained transformer-based language model, specifically a DistilBERT model, to generate embeddings for text documents.

In [ ]:
!pip install openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.0 MB/s eta 0:00:00


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
import numpy as np
import torch
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModel
import os
import hashlib

# Load a smaller subset of the 20 Newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
documents, labels = newsgroups.data[:1000], newsgroups.target[:1000]

# Load a smaller and more efficient transformer model
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate embeddings for text documents
def generate_embeddings(texts, batch_size=32):
    cache_dir = '.embeddings_cache'
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, hashlib.md5('\n'.join(texts).encode('utf-8')).hexdigest() + '.npy')
    if os.path.exists(cache_path):
        embeddings = np.load(cache_path)
    else:
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            input_ids = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512)['input_ids']
            with torch.no_grad():
                outputs = model(input_ids.to(device))
            embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())
        embeddings = np.concatenate(embeddings)
        np.save(cache_path, embeddings)
    return embeddings

# Generate embeddings for the text documents
X = generate_embeddings(documents)
y = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Make predictions and evaluate the classifier
y_pred = clf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.45      0.50        11
           1       0.11      0.20      0.14        10
           2       0.45      0.56      0.50         9
           3       0.50      0.24      0.32        17
           4       0.00      0.00      0.00         7
           5       0.62      0.62      0.62        13
           6       0.50      0.62      0.56         8
           7       0.50      0.18      0.27        11
           8       0.18      0.55      0.27        11
           9       0.50      0.33      0.40        12
          10       0.60      0.46      0.52        13
          11       0.20      0.30      0.24        10
          12       0.23      0.43      0.30         7
          13       0.67      0.40      0.50        10
          14       1.00      0.42      0.59        12
          15       0.75      0.75      0.75         8
          16       0.17      0.11      0.13         9
   

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
import torch
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from transformers import AutoTokenizer, AutoModel
import os
import hashlib

# Load a smaller subset of the 20 Newsgroups dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
documents, labels = newsgroups.data[:1000], newsgroups.target[:1000]

# Load a smaller and more efficient transformer model
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate embeddings for text documents
def generate_embeddings(texts, batch_size=32):
    cache_dir = '.embeddings_cache'
    os.makedirs(cache_dir, exist_ok=True)
    cache_path = os.path.join(cache_dir, hashlib.md5('\n'.join(texts).encode('utf-8')).hexdigest() + '.npy')
    if os.path.exists(cache_path):
        embeddings = np.load(cache_path)
    else:
        embeddings = []
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            input_ids = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512)['input_ids']
            with torch.no_grad():
                outputs = model(input_ids.to(device))
            embeddings.append(outputs.last_hidden_state.mean(dim=1).cpu().numpy())
        embeddings = np.concatenate(embeddings)
        np.save(cache_path, embeddings)
    return embeddings

# Generate embeddings for the text documents
X = generate_embeddings(documents)
y = np.array(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the SVM classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

# Make predictions and evaluate the classifier
y_pred = clf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.45      0.50        11
           1       0.11      0.20      0.14        10
           2       0.45      0.56      0.50         9
           3       0.50      0.24      0.32        17
           4       0.00      0.00      0.00         7
           5       0.62      0.62      0.62        13
           6       0.50      0.62      0.56         8
           7       0.50      0.18      0.27        11
           8       0.18      0.55      0.27        11
           9       0.50      0.33      0.40        12
          10       0.60      0.46      0.52        13
          11       0.20      0.30      0.24        10
          12       0.23      0.43      0.30         7
          13       0.67      0.40      0.50        10
          14       1.00      0.42      0.59        12
          15       0.75      0.75      0.75         8
          16       0.17      0.11      0.13         9
   

The above app is a text classification model that uses a pre-trained transformer-based language model, specifically a DistilBERT model, to generate embeddings for text documents. These embeddings are then used as features to train a support vector machine (SVM) classifier to classify the text documents into categories.